In [1]:
import qppy as qp
import numpy as np
from plotly import express as px
import pandas as pd

In [2]:
I = qp.Instance()
I.add_variable(qp.Variable('x', None, 0.9))
I.add_quadratic_objective_term('x', 'x', 2.0)
I.add_variable(qp.Variable('y', -0.4, 0.6))
I.add_quadratic_objective_term('y', 'x', -3.0)
I.add_linear_objective_term('y', -1.0)

c1 = qp.LinearConstraint(name='c1')
c1.add_term('x', 1.0)
c1.add_term('y', 1.5)
c1.set_relation(qp.ConstraintRelation.LESS_THAN)
c1.set_rhs(0.5)
I.add_constraint(
    c1
)

print(I)

I.compile()

qppy.Instance with 2 variables and 4 constraints.
minimize
	+2.0*x² -3.0*y*x
	-1.0*y
subject to
	c1: 1.0*x + 1.5*y <= 0.5
	x in [inf, 0.9]
	y in [-0.4, 0.6]
	x in R
	y in R


In [3]:
x1 = I.solution_from_dict({'x': 1.0, 'y': -0.1})
print(f"x1 = {x1}")
print(f"correct = {2.0*x1[0]**2 -3.0*x1[1]*x1[0] -1.0*x1[1]}")
print(f"objective = {I.objective(x1)}")
print(f"feasible: {I.is_feasible(x1)}")

x2 = I.solution_from_dict({'x': -101.0, 'y': 0.1})
print(f"\nx2 = {x2}")
print(f"correct = {2.0*x2[0]**2 -3.0*x2[1]*x2[0] -1.0*x2[1]}")
print(f"objective = {I.objective(x2)}")
print(f"feasible: {I.is_feasible(x2)}")

x1 = [ 1.  -0.1]
correct = 2.4
objective = 2.4
feasible: False

x2 = [-1.01e+02  1.00e-01]
correct = 20432.2
objective = 20432.200000000004
feasible: True


In [4]:
np.random.seed(0)

solutions = [{'x': x, 'y': y} for x, y in zip(2*np.random.rand(1000)-1, 2*np.random.rand(1000)-1)]
X = I.vectorized_solution_from_dict(solutions)

In [5]:
%%timeit
o = I.vectorized_objective(X)

37.9 μs ± 407 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%%timeit
[I.objective(x) for x in X]

466 μs ± 3.36 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
df = pd.DataFrame.from_records(solutions)
df['objective'] = I.vectorized_objective(X)
df['feasible'] = I.vectorized_is_feasible(X)
display(df)
px.scatter_3d(df, x='x', y='y', z='objective', color='feasible', opacity=0.5)

,x,y,objective,feasible
0,0.097627,0.185761,-0.221104,True
1,0.430379,-0.979873,2.615473,False
2,0.205527,-0.048348,0.162640,True
3,0.089766,0.417541,-0.513868,False
4,-0.152690,-0.912049,0.540894,False
...,...,...,...,...
995,-0.804647,0.582563,2.118625,True
996,0.029844,0.442396,-0.480224,False
997,0.876824,-0.039784,1.682077,False
998,-0.542707,0.287728,0.769790,True
